In [1]:
from ttml.tt_cross import estimator_to_tt_cross
import numpy as np

def f(x):
    return np.sin(np.sum(x, axis=1))

# Same thresholds for every feature: [0,0.2,0.4,0.6,0.8]
thresholds = [np.linspace(0, 1, 11)] * 5

tt = estimator_to_tt_cross(f, thresholds)
tt.gather(np.array([[2, 2, 2, 2, 2]])) - np.sin(1)

array([8.8817842e-16])

In [3]:
tt.sing_vals()

[array([2.62706082e+02, 1.10057236e+02, 2.86676345e-14, 2.41656203e-14,
        1.97694406e-14]),
 array([2.50489642e+02, 1.35580309e+02, 3.79018148e-14, 2.62751171e-14,
        1.44061788e-14]),
 array([2.50489642e+02, 1.35580309e+02, 8.86371342e-14, 5.93783970e-14,
        2.95181315e-14]),
 array([2.62706082e+02, 1.10057236e+02, 5.49963548e-14, 3.80107010e-14,
        2.08506835e-14])]

In [5]:
tt2 = estimator_to_tt_cross(f, thresholds, max_rank=2)

tt2

<TensorTrain of order 5 with outer dimensions (11, 11, 11, 11, 11), TT-rank (2, 2, 2, 2), and orthogonalized at mode 4>

In [6]:
(tt - tt2).norm()

4.924250918078487e-13

In [7]:
from sklearn.ensemble import RandomForestRegressor

X = np.random.normal(size=(1000, 5))
y = np.exp(np.sum(X, axis=1))

forest = RandomForestRegressor()
forest.fit(X, y)

thresholds = [np.linspace(0, 1, 11)] * 5

tt = estimator_to_tt_cross(forest.predict, thresholds, max_rank=2)
tt

<TensorTrain of order 5 with outer dimensions (11, 11, 11, 11, 11), TT-rank (2, 2, 2, 2), and orthogonalized at mode 4>

In [8]:
tt = estimator_to_tt_cross(forest.predict, thresholds, method='dmrg')